In [1]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import neuropythy as ny
from scipy import stats
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# personal import 
from math_utils import weighted_regression, weighted_nan_percentile, weighted_nan_median



In [2]:
# Setings 
rois = ['V1', 'V2', 'V3']
max_ecc = 8
roi_colors = {'V1': 'rgb(243, 231, 155)', 
              'V2': 'rgb(250, 196, 132)', 
              'V3': 'rgb(248, 160, 126)'}

In [3]:
# Load data
tsv_dir = '/home/jovyan/projects/pRF-project_NH2025/data/tsv/group'
tsv_fn = '{}/final_group_df.tsv'.format(tsv_dir)
group_bin_df = pd.read_table(tsv_fn, sep="\t")


In [27]:
rows= 1
cols = len(rois)

fig = make_subplots(rows=rows, cols=cols)

for n_roi, roi in enumerate(rois):
    roi_color = roi_colors[roi]
    roi_color_opac = f"rgba{roi_color[3:-1]}, 0.15)"
    df_roi = group_bin_df.loc[group_bin_df['roi']==roi]
    
    # weighted_regression (harvey equation)
    slope, intercept = weighted_regression(df_roi['prf_ecc_bins'], 
                                           df_roi['prf_pcm_bins_median'], 
                                           df_roi['prf_r2_bins_median'], 
                                           model='pcm')
    
    # weighted_regression CI (harvey equation)
    slope_lower, intercept_lower = weighted_regression(df_roi['prf_ecc_bins'], 
                                                       df_roi['prf_pcm_bins_median_lower_bound'], 
                                                       df_roi['prf_r2_bins_median'], 
                                                       model='pcm')

    slope_upper, intercept_upper = weighted_regression(df_roi['prf_ecc_bins'], 
                                                       df_roi['prf_pcm_bins_median_upper_bound'], 
                                                       df_roi['prf_r2_bins_median'], 
                                                       model='pcm')
    
    # Compute R2 between Harvey model cm and data cm
    harvey_predict_CM = 1 / (slope * df_roi['prf_ecc_bins'] + intercept)
    result_harvey = stats.linregress(harvey_predict_CM, df_roi['prf_pcm_bins_median'])
    harvey_r2 = result_harvey.rvalue ** 2

    # Compute R2 between Horton model cm and data cm
    horton_x_line_continuous = np.linspace(0,8,100)
    horton_line_continuous = 17.3/(horton_x_line_continuous + 0.75)

    # Compute R2 between Horton model cm and data cm
    horton_x = df_roi['prf_ecc_bins']
    horton_line = 17.3/(horton_x + 0.75)
    
    res_horton = stats.linregress(horton_line, df_roi['prf_pcm_bins_median'])
    horton_model_r2 = res_horton.rvalue ** 2

    # Create Harvey model line
    model_x = np.linspace(0,10,100)
    model_y = 1 / (slope * model_x + intercept)

    model_y_lower_bound = 1 / (slope_lower * model_x + intercept_lower)
    model_y_upper_bound = 1 / (slope_upper * model_x + intercept_upper)
    

    
    # Horton Model
    fig.add_trace(go.Scatter(x=horton_x_line_continuous, 
                             y=horton_line_continuous, 
                             mode='lines', 
                             name='Horton_model, R2={:.2f}'.format(horton_model_r2), 
                             marker=dict(color='black', 
                                         symbol='circle', 
                                         size=2), 
                             showlegend=False), 
                  row=1, col=n_roi+1)
    
    #  Harvey model 
    fig.add_trace(go.Scatter(x=model_x, 
                             y=model_y, 
                             mode='lines', 
                             name=f'Harvey model, R2={harvey_r2:.2f}', 
                             line=dict(color=roi_colors[roi], width=2), 
                             showlegend=False), 
                  row=1, col=n_roi+1)

    # Bins
    fig.add_trace(go.Scatter(x=df_roi['prf_ecc_bins'], 
                             y=df_roi['prf_pcm_bins_median'], 
                             mode='markers', 
                             marker=dict(color=roi_colors[roi], 
                                         symbol='square', 
                                         size=5), 
                             error_y=dict(type='data', 
                                          array=df_roi['prf_pcm_bins_median_upper_bound_ci'] - df_roi['prf_pcm_bins_median'], 
                                          arrayminus=df_roi['prf_pcm_bins_median'] - df_roi['prf_pcm_bins_median_lower_bound_ci'], 
                                          visible=True, 
                                          thickness=1, 
                                          width=0, 
                                          color=roi_color), 
                             showlegend=False), 
                  row=1, col=n_roi+1)

    # Error area
    fig.add_trace(go.Scatter(x=np.concatenate([model_x, model_x[::-1]]), 
                             y=np.concatenate([list(model_y_upper_bound), list(model_y_lower_bound[::-1])]), 
                             mode='lines', fill='toself', fillcolor=roi_color_opac, 
                             line=dict(color=roi_color_opac, width=0), showlegend=False), 
                  row=1, col=n_roi+1)

    # Add legend
    annotation = go.layout.Annotation(x=6, y=40, 
                                      text='{},<br>Harvey R<sup>2</sup>={:.2f} <br>Horton R<sup>2</sup>={:.2f}'.format(roi, harvey_r2, horton_model_r2), 
                                      xanchor='left',
                                      showarrow=False, 
                                      font=dict(size=14, 
                                                color=roi_colors[roi]))
    fig.add_annotation(annotation, row=1, col=n_roi+1)

    fig.update_yaxes(title_text='pRF cortical magn. (mm/dva)', row=1, col=1)
    fig.update_xaxes(title_text='pRF eccentricity (dva)', range=[0, max_ecc], showline=True, row=1, col=n_roi+1)

    # fig.update_layout(showlegend=False, 
    #                   height=1000, 
    #                   width=1800, 
    #                   template='simple_white')



fig.update_yaxes(range=[0, 40], showline=True)
fig.update_layout(
    showlegend=False, 
    height=600,
    width=1800,
    template='simple_white'
)
# fig.show()
fig.write_image('/home/jovyan/projects/pRF-project_NH2025/figures/group_fig.pdf',height=600, width=1800,)


In [5]:
df_roi['prf_ecc_bins'].max()

5.691385269165039